<span style="font-size:32px;"><strong>Date Alignment</strong></span>


In [1]:
import nltk
nltk.download('vader_lexicon')
import pandas as pd
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer


news_df = pd.read_csv("C:/Users/User/Desktop/10/raw_analyst_ratings.csv")
stock_df = pd.concat([
    pd.read_csv('C:/Users/User/Desktop/10/data/AAPL_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/AMZN_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/GOOG_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/META_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/MSFT_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/NVDA_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/TSLA_historical_data.csv')
], ignore_index=True)

print("News DataFrame - Data types:")
print(news_df.dtypes)
print("\nNews DataFrame - First few rows:")
print(news_df.head())

print("\nStock DataFrame - Data types:")
print(stock_df.dtypes)
print("\nStock DataFrame - First few rows:")
print(stock_df.head())

news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
print(f"\nNews date column type: {news_df['date'].dtype}")
print(f"News date column first few values: {news_df['date'].head()}")

stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce')
print(f"\nStock date column type: {stock_df['Date'].dtype}")
print(f"Stock date column first few values: {stock_df['Date'].head()}")

print(f"\nNon-datetime values in news_df['date']: {news_df[news_df['date'].isna()]}")

news_df
 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


News DataFrame - Data types:
Unnamed: 0     int64
headline      object
url           object
publisher     object
date          object
stock         object
dtype: object

News DataFrame - First few rows:
   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://ww

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A
...,...,...,...,...,...,...
1407323,1413844,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,2011-08-29 00:00:00,ZX
1407324,1413845,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,2011-06-22 00:00:00,ZX
1407325,1413846,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,2011-06-21 00:00:00,ZX
1407326,1413847,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,2011-06-21 00:00:00,ZX


<span style="font-size:32px;"><strong>Sentiment Analysis</strong></span>

In [ ]:
def analyze_sentiment_textblob(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

news_df['sentiment_textblob'] = news_df['headline'].apply(analyze_sentiment_textblob)

print(news_df[['headline', 'sentiment_textblob']].head())

<span style="font-size:32px;"><strong>Daily Stock Returns</strong></span>

In [6]:
aapl_df = pd.read_csv('C:/Users/User/Desktop/10/data/AAPL_historical_data.csv')
aapl_df['Stock'] = 'AAPL' 

amzn_df = pd.read_csv('C:/Users/User/Desktop/10/data/AMZN_historical_data.csv')
amzn_df['Stock'] = 'AMZN' 
goog_df = pd.read_csv('C:/Users/User/Desktop/10/data/GOOG_historical_data.csv')
goog_df['Stock'] = 'GOOG' 
meta_df = pd.read_csv('C:/Users/User/Desktop/10/data/META_historical_data.csv')
meta_df['Stock'] = 'META' 

msft_df = pd.read_csv('C:/Users/User/Desktop/10/data/MSFT_historical_data.csv')
msft_df['Stock'] = 'MSFT'  

nvda_df = pd.read_csv('C:/Users/User/Desktop/10/data/NVDA_historical_data.csv')
nvda_df['Stock'] = 'NVDA'  

tsla_df = pd.read_csv('C:/Users/User/Desktop/10/data/TSLA_historical_data.csv')
tsla_df['Stock'] = 'TSLA'  

stock_df = pd.concat([aapl_df, amzn_df, goog_df, meta_df, msft_df, nvda_df, tsla_df], ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce')

stock_df = stock_df.sort_values(by=['Stock', 'Date'])

stock_df['Daily_Return'] = stock_df.groupby('Stock')['Close'].pct_change() * 100

stock_df['Daily_Return'] = stock_df['Daily_Return'].fillna(0)

print("Stock DataFrame with Daily Returns:")
print(stock_df[['Date', 'Stock', 'Close', 'Daily_Return']].head(10))


Stock DataFrame with Daily Returns:
        Date Stock     Close  Daily_Return
0 1980-12-12  AAPL  0.128348      0.000000
1 1980-12-15  AAPL  0.121652     -5.217061
2 1980-12-16  AAPL  0.112723     -7.339788
3 1980-12-17  AAPL  0.115513      2.475091
4 1980-12-18  AAPL  0.118862      2.899246
5 1980-12-19  AAPL  0.126116      6.102867
6 1980-12-22  AAPL  0.132254      4.866958
7 1980-12-23  AAPL  0.137835      4.219904
8 1980-12-24  AAPL  0.145089      5.262818
9 1980-12-26  AAPL  0.158482      9.230886
